In [2]:
!pip install -qq mediapipe --user

ERROR: Could not find a version that satisfies the requirement mediapipe (from versions: none)
ERROR: No matching distribution found for mediapipe


In [1]:
import mediapipe as mp
import cv2

# MediaPipe Pose 설정
pose = mp.solutions.pose.Pose(
    static_image_mode=False, # 입력 이미지가 정적 이미지인지, 비디오 스트림 프레임인지 설정(False인 경우 비디오 스트림으로 처리)
    model_complexity=1, # 0,1,2 의 순으로 정확도가 올라가는 대신 속도가 느림
    smooth_landmarks=True, # 프레임간의 랜드 마크 위치를 부드럽게 처리할 지 여부 결정
    enable_segmentation=True, # 포즈 검출과 인물 마스크를 생성할지 여부 결정. 배경 제거 등의 용도로 사용
    smooth_segmentation=True, # 인물 마스크의 경계를 부드럽게 처리
    min_detection_confidence=0.5, # 사람 검출 최소 신뢰도 임계값
    min_tracking_confidence=0.5 # 포즈 추적의 신뢰도 임계값
)

# 그리기 함수 초기화
mp_drawing = mp.solutions.drawing_utils

# 웹캠 캡처
cap = cv2.VideoCapture(0)

while True:
    # 이미지 읽기
    ret, image = cap.read()
    if not ret:
        break

    # 이미지 변환
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # 포즈 추정
    results = pose.process(image)

    # 추정된 포즈 정보 확인
    if results.pose_landmarks:

        # 연결된 랜드마크 시각화
        mp_drawing.draw_landmarks(
            image, results.pose_landmarks, mp.solutions.pose.POSE_CONNECTIONS)

    # 세그멘테이션 마스크 추출 및 배경 제거
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    if results.segmentation_mask is not None:
        # 세그멘테이션 마스크를 이미지 크기에 맞게 조정
        segmentation_mask = results.segmentation_mask
        segmentation_mask = (segmentation_mask > 0.1)  # 임계값 적용
        segmentation_mask = segmentation_mask.astype(float)
        
        # 세그멘테이션 마스크를 3채널로 복제하여 컬러 이미지와 동일한 차원으로 만듦
        mask_three_channel = cv2.merge([segmentation_mask, segmentation_mask, segmentation_mask])
        
        # 마스크를 적용하여 배경을 제거한 이미지 생성
        foreground = cv2.multiply(mask_three_channel, image.astype(float))
        
        cv2.imshow('Foreground', foreground.astype('uint8'))

    # 이미지 출력
    cv2.imshow('MediaPipe Pose', image)

    # 키 입력 처리
    if cv2.waitKey(5) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

ModuleNotFoundError: No module named 'mediapipe'

In [21]:
import mediapipe as mp
import cv2

background_image = cv2.imread("dog.png")

# Check if the background image is loaded successfully
if background_image is None:
    print("Failed to load the background image. Please check the file path.")
    exit()

# MediaPipe Pose 설정
pose = mp.solutions.pose.Pose(
    static_image_mode=False, # 입력 이미지가 정적 이미지인지, 비디오 스트림 프레임인지 설정(False인 경우 비디오 스트림으로 처리)
    model_complexity=1, # 0,1,2 의 순으로 정확도가 올라가는 대신 속도가 느림
    smooth_landmarks=True, # 프레임간의 랜드 마크 위치를 부드럽게 처리할 지 여부 결정
    enable_segmentation=True, # 포즈 검출과 인물 마스크를 생성할지 여부 결정. 배경 제거 등의 용도로 사용
    smooth_segmentation=True, # 인물 마스크의 경계를 부드럽게 처리
    min_detection_confidence=0.5, # 사람 검출 최소 신뢰도 임계값
    min_tracking_confidence=0.5 # 포즈 추적의 신뢰도 임계값
)

# 그리기 함수 초기화
mp_drawing = mp.solutions.drawing_utils

# 웹캠 캡처
cap = cv2.VideoCapture(0)

while True:
    # 이미지 읽기
    ret, image = cap.read()
    if not ret:
        break

    # 이미지 변환
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # 포즈 추정
    results = pose.process(image)

    # 추정된 포즈 정보 확인
    if results.pose_landmarks:

        # 연결된 랜드마크 시각화
        mp_drawing.draw_landmarks(
            image, results.pose_landmarks, mp.solutions.pose.POSE_CONNECTIONS)

    # 세그멘테이션 마스크 추출 및 배경 제거
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    if results.segmentation_mask is not None:
        # 세그멘테이션 마스크를 이미지 크기에 맞게 조정
        segmentation_mask = results.segmentation_mask
        segmentation_mask = (segmentation_mask > 0.1)  # 임계값 적용
        segmentation_mask = segmentation_mask.astype(float)
        
        background_image_resized = cv2.resize(background_image, (image.shape[1], image.shape[0]))
        new_background = cv2.bitwise_and(background_image_resized, background_image_resized, mask=segmentation_mask.astype('uint8'))

        image += new_background

        # 세그멘테이션 마스크를 3채널로 복제하여 컬러 이미지와 동일한 차원으로 만듦
        #mask_three_channel = cv2.merge([segmentation_mask, segmentation_mask, segmentation_mask])
        
        # 마스크를 적용하여 배경을 제거한 이미지 생성
        #foreground = cv2.multiply(mask_three_channel, image.astype(float))
        
        cv2.imshow('Foreground', foreground.astype('uint8'))

    # 이미지 출력
    cv2.imshow('MediaPipe Pose', image)

    # 키 입력 처리
    if cv2.waitKey(5) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [20]:
import mediapipe as mp
import cv2

# Load the background image
background_image = cv2.imread("dog.png")

# Check if the background image is loaded successfully
if background_image is None:
    print("Failed to load the background image. Please check the file path.")
    exit()

# MediaPipe Pose setup
pose = mp.solutions.pose.Pose(
    static_image_mode=False,
    model_complexity=1,
    smooth_landmarks=True,
    enable_segmentation=True,
    smooth_segmentation=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

# Initialize drawing function
mp_drawing = mp.solutions.drawing_utils

# Webcam capture
cap = cv2.VideoCapture(0)

while True:
    # Read the frame
    ret, image = cap.read()
    if not ret:
        break

    # Convert the image to RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Pose estimation
    results = pose.process(image_rgb)

    # Check if pose landmarks are detected
    if results.pose_landmarks:
        # Visualize the landmarks
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp.solutions.pose.POSE_CONNECTIONS)

    # Convert the image back to BGR
    #image_bgr = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    image_bgr = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Extract segmentation mask and replace the background
    if results.segmentation_mask is not None:
        # Resize the segmentation mask to match the image dimensions
        segmentation_mask = cv2.resize(results.segmentation_mask, (image_bgr.shape[1], image_bgr.shape[0]))

        # Invert the mask (since we want the background, not the person)
        segmentation_mask = 1 - segmentation_mask

        # Resize the background image to match the image dimensions
        background_image_resized = cv2.resize(background_image, (image_bgr.shape[1], image_bgr.shape[0]))

        # Apply the mask to the background image
        new_background = cv2.bitwise_and(background_image_resized, background_image_resized, mask=segmentation_mask.astype('uint8'))

        # Apply the inverse mask to the original image
        #image = cv2.bitwise_and(image_bgr, image_bgr, mask=segmentation_mask.astype('uint8'))

        # Combine the masked images
        image += new_background

    # Display the image
    cv2.imshow('MediaPipe Pose', image)

    # Check for key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close the windows
cap.release()
cv2.destroyAllWindows()

In [15]:
import mediapipe as mp
import cv2

# Load the background image
background_image = cv2.imread("dog.png")

# Check if the background image is loaded successfully
if background_image is None:
    print("Failed to load the background image. Please check the file path.")
    exit()

# MediaPipe Pose setup
pose = mp.solutions.pose.Pose(
    static_image_mode=False,
    model_complexity=1,
    smooth_landmarks=True,
    enable_segmentation=True,
    smooth_segmentation=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

# Initialize drawing function
mp_drawing = mp.solutions.drawing_utils

# Webcam capture
cap = cv2.VideoCapture(0)

while True:
    # Read the frame
    ret, image = cap.read()
    if not ret:
        break

    # Convert the image to RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Pose estimation
    results = pose.process(image_rgb)

    # Check if pose landmarks are detected
    if results.pose_landmarks:
        # Visualize the landmarks
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp.solutions.pose.POSE_CONNECTIONS)

    # Convert the image back to BGR
    image_bgr = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # Extract segmentation mask and replace the background
    if results.segmentation_mask is not None:
        # Resize the segmentation mask to match the image dimensions
        segmentation_mask = cv2.resize(results.segmentation_mask, (image_bgr.shape[1], image_bgr.shape[0]))

        # Invert the mask (since we want the background, not the person)
        segmentation_mask = 1 - segmentation_mask

        # Resize the background image to match the image dimensions
        background_image_resized = cv2.resize(background_image, (image_bgr.shape[1], image_bgr.shape[0]))

        # Extract original background color
        original_background = cv2.bitwise_and(background_image_resized, background_image_resized, mask=cv2.bitwise_not(segmentation_mask))

        # Apply the mask to the background image
        new_background = cv2.bitwise_and(background_image_resized, background_image_resized, mask=segmentation_mask)

        # Apply the inverse mask to the original image
        image = cv2.bitwise_and(image_bgr, image_bgr, mask=cv2.bitwise_not(segmentation_mask))

        # Combine the masked images
        image += new_background

    # Display the image
    cv2.imshow('MediaPipe Pose', image)

    # Check for key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close the windows
cap.release()
cv2.destroyAllWindows()

error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\core\src\arithm.cpp:230: error: (-215:Assertion failed) (mtype == CV_8U || mtype == CV_8S) && _mask.sameSize(*psrc1) in function 'cv::binary_op'
